# How to create agents with configuration

One of the benefits of LangGraph API is that it lets you create agents with different configurations.
This is useful when you want to:

- Define a cognitive architecture once as a LangGraph
- Let that LangGraph be configurable across some attributes (for example, system message or LLM to use)
- Let users create agents with arbitrary configurations, save them, and then use them in the future

In this guide we will show how to do that for the default agent we have built in.

If you look at the agent we defined, you can see that inside the `call_model` node we have created the model based on some configuration. That node looks like:

```python
def call_model(state, config):
    messages = state["messages"]
    model_name = config.get('configurable', {}).get("model_name", "anthropic")
    model = _get_model(model_name)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}
```

We are looking inside the config for a `model_name` parameter (which defaults to `anthropic` if none is found).
That means that by default we are using Anthropic as our model provider.
In this example we will see an example of how to create an example agent that is configured to use OpenAI.

In [3]:
from langgraph_sdk import get_client

client = get_client()

assistant = await client.assistants.create(graph_id="agent", config={"configurable": {"model_name": "openai"}})

We can see that this assistant has saved the config

In [4]:
assistant

{'assistant_id': '4048284c-af4c-4712-bfa9-0016bb47868d',
 'graph_id': 'agent',
 'config': {'configurable': {'model_name': 'openai'}},
 'created_at': '2024-05-21T16:11:55.242301+00:00',
 'updated_at': '2024-05-21T16:11:55.242301+00:00',
 'metadata': {}}

In [6]:
thread = await client.threads.create()
input = {"messages": [{"role": "user", "content": "who made you?"}]}
async for event in client.runs.stream(thread['thread_id'], assistant['assistant_id'], input=input):
    print(event)

StreamPart(event='metadata', data={'run_id': '107ae507-f370-4db2-9e66-3b909df3dbc8'})
StreamPart(event='values', data={'messages': [{'role': 'user', 'content': 'who made you?'}]})
StreamPart(event='values', data={'messages': [{'content': 'who made you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '59feeef8-b02b-4933-aee1-8b1c549a6e9b', 'example': False}, {'content': 'I was created by OpenAI, a research organization focused on developing and advancing artificial intelligence technology.', 'additional_kwargs': {}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-626971fb-cf61-4b1c-8180-056dc572c6af', 'example': False, 'tool_calls': [], 'invalid_tool_calls': []}]})
StreamPart(event='end', data=None)
